# Taller de redes neuronales aplicadas a espectros estelares

En este notebook se va a entrenar una red neuronal utilizando la librería sklearn. En concreto, la red neuronal es un objeto de la clase [sklearn.neural_network.MLPRegresor](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html). Posteriormente se utilizará la red neuronal entrenada para inferir parámetros astrofísicos de un conjunto de estrellas cuyos espectros fueron medidos con [RAVE](https://www.rave-survey.org/).

Para entrenar dicha red se han generado espectros sintéticos que se han guardado en el fichero *grid_school_v2_8420_8785_003899999999998727.pkl*. Dentro del fichero está guardado, además de los espectros, la temperatura efectiva, el logaritmo de la gravedad y la metalicidad que corresponde a cada espectro; todo ello en formato tabla como un objeto [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html). 

Para comenzar importamos las librerías necesarias:

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle

from warnings import simplefilter
simplefilter(action="ignore",category=FutureWarning)

%matplotlib inline

Además de las librerías que se han mencionado, se han importado las funciones [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) y [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) que serán de utilidad para trabajar con los datos de entrenamiento

In [ ]:
# Código auxiliar: aunque no es necesario, se definen las longitudes de onda tanto de las simulaciones como de las observaciones
wlen = np.arange(842,878.5,0.03899999999998727) 

## 1 Preparación de los datos



### 1.1 Lectura de las simulaciones

Primero hay que cargar las simulaciones. Para ello hay que utilizar la función *pandas.read_pickle* que acepta como argumento el nombre del fichero a leer y devuelve un *DataFrame* (nota: para llamar a las funciones de pandas hay que usar pd en lugar de pandas, ya que en la cabecera se ha llamado a la instrucción 
```
import pandas as pd
```


In [ ]:
grid = ____ # rellenar

Una vez leído, la variable *grid* tiene estructura de tabla, para saber qué columnas hay almacenadas dentro se puede llamar al atributo *columns*.


In [ ]:
cols = ____  # rellenar
print(____ ) #rellenar 

Si se quieren ver las primeras filas de la tabla se puede llamar al método *head()* que tiene todo *DataFrame*. El resultado debería ser igual a

|  |   teff | logg |  m_h |                                          spectrum|
|--|--------|------|------|--------------------------------------------------|
|0 | 3000.0 |  0.0 |-3.00 | [0.9997522849383049, 0.9998009367060697, 0.999...|
|1 | 3000.0 |  0.0 |-2.75 | [0.9997123698112049, 0.9997728740768044, 0.999...|
|2 | 3000.0 |  0.0 |-2.50 | [0.999568427270208, 0.9996652039896338, 0.9997...|
|3 | 3000.0 |  0.0 |-2.25 | [0.9993215807872953, 0.9994811204761738, 0.999...|
|4 | 3000.0 |  0.0 |-2.00 | [0.9991882596346863, 0.9993762569755674, 0.999...|

Por otro lado, si queremos ver una columna en concreto (por ejemplo, la columna j), se puede acceder a ella mediante 
```
nombreDataFrame.iloc[j]
```

In [ ]:
# Rellenar la celda para mostrar las primeras filas almacenadas en grid


### 1.2 Visualización de las simulaciones

En la celda siguiente hay un ejemplo en el que se pinta el espectro almacenado en la primera fila del *DataFrame*.

In [ ]:
idx = 0
spec = grid["spectrum"].iloc[idx]
teff = grid["teff"].iloc[idx]
logg = grid["logg"].iloc[idx]
mh = grid["m_h"].iloc[idx]
label_spec = "Teff = {} log(g) = {} [m/H] = {}".format(teff1,logg1,mh1) 

plt.figure(figsize=(16,8))
plt.plot(wlen,spec,label=label_spec)
plt.xlabel(r"$\lambda /nm$",size=20)
plt.ylabel(r"Normalized flux",size=20)
plt.legend()

En la misma figura se pueden representar varios espectros.

In [ ]:
idx1 = ____ # elige una fila para el primer espectro
idx2 = ____ # elige otra fila para el segundo espectro

spec1 = ____ # rellenar
teff1 = ____ # rellenar
logg1 = ____ # rellenar
mh1 = ____ # rellenar
label_spec1 = ____ # rellenar

# rellenar para definir las variables del segundo espectro
    

# rellenar para dibujar los dos espectros


Además, se pueden seleccionar las columnas guardadas en el *DataFrame* de *pandas* simplemente poniendo entre corchetes los nombres de las variables

In [ ]:
teffs = grid["____"] # rellenar para escoger la columna de las temperaturas
loggs = grid["____"] # columna de las gravedades
mhs = grid["____"] # columna de las metalicidades

plt.figure(figsize=(21,8))
plt.subplot(131)
plt.hist(teffs,bins=100)
plt.xlabel(r"$T_{eff}$")
plt.subplot(132)
plt.hist(loggs,bins=100)
plt.xlabel(r"$log(g)$")
plt.subplot(133)
plt.hist(mhs,bins=100)
plt.xlabel(r"$[M/H]$")

### 1.3 Preparación de las simulaciones

Una vez observados y entendidos los datos simulados con los que se puede trabajar es cuando se van a preparar para poder usarlos en el entrenamiento de la red neuronal.

La red es lo que se llama un perceptrón totalmente conectado y queremos entrenarlo para que lea un espectro y como resultado nos diga qué parámetros astrofísicos (temperatura efectiva, logaritmo de la gravedad y metalicidad) tiene la estrella que produce dicho espectro. Por tanto, los valores de entrada de la red serán los valores del espectro para las distintas longitudes de onda y los valores de salida serán tres: temperatura, gravedad y metalicidad.

In [ ]:
y = grid[["teff","logg","m_h"]].values
x = grid[""].values # rellenar 
x = np.array([xx.astype(float) for xx in x]) # este paso es necesario debido a la implementación de sklearn

A la hora de ayudar al programa de minimización a encontrar el mínimo es útil escalar las variables de manera que el rango entre ellas sea aproximadamente igual. Para esto vamos a utilizar el objeto [MinMaxScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) de la librería *sklearn*

In [ ]:
scaler = MinMaxScaler()
ys = scaler.fit_transform(____) # rellenar con la variable que se quiere escalar

Una vez preprocesados los datos sintéticos hay que decidir qué datos se utilizarán para entrenar y cuáles de ellos (el resto) para comprobar. Ésto se logra fácilmente con la función *train_test_split* que recibe como primer y segundo argumentos dos *arrays* que va a separar en dos conjuntos cada uno de ellos, el conjunto de entrenamiento (*train*) y el conjunto de comprobación (*test*). Además, admite otro argumento mediante la [clave](https://docs.python.org/3/glossary.html#term-argument) *test_size* cuyo valor va de 0 a 1 y que define qué porcentaje de elementos de los arrays se utilizarán para entrenar y qué porcentaje se utilizarán para comprobar. Los valores típicos suelen ser 0.1, 0.15 y 0.2.


In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(____,____,test_size=____) # rellenar

In [ ]:
# rellenar la celda para mostrar tres gráficas con las variables de entrada que se usarán para entrenar
# nota: en una celda anterior se mostraron gráficas con los parámetros astrofísicos de entrad, ésta puede ser de ayuda


A la vista de los gráficos anteriores, ¿se distribuyen los datos de entrenamiento como queremos?

## 2 Construcción de la red

La definición de la red se hace a través de parámetros e hiperparámetros. Algunos de ellos hacen referencia a la arquitectura de la propia red mientras que otros definen la estrategia a seguir para la optimización de los parámetros de la red.

### 2.1 Parámetros de la red

Los parámtros que definen la red son las funciones de activación así como el número de capas 
ocultas y de neuronas por capa.

Cuando se trabaja con *MLPRegressor* esto se hace con los argumentos *activation* y *hidden_layer_sizes*.

- activation: función de activación de las neuronas. Admite un sólo valor entre "logistic", "linear", "tanh" y "relu" 
- hidden_layer_sizes: define el número de capas ocultas y el número de neuronas que hay en cada capa. El argumento es una tupla cuya longitud denota el número de capas oculatas y cuyos valores marcan el número de neuronas en las capas. Así, por ejemplo, una neurona con 2 capas ocultas y tres neuronas en cada capa vendría definida por
``` 
hidden_layer_sizes=(3,3)
```

In [ ]:
activation = ____ # elegir una función de activación
hidden_layer_sizes = ____ # elegir el número de capas ocultas así como las neuronas en cada capa
alpha = ____ # término de regularización

### 2.2 Parámetros para el optimizador

Una vez definida la red quedan por definir tanto el optimizador como los parámetros que se 
pasarán a éste.

El primero de ellos es el optimizador que se va a utilizar ("lbfgs", "sgd" o "adam"), mientras que los otros dos hacen referencia al desplazamiento de la solución que se obtiene tras cada iteración. Estos argumentos vienen especificados por *learning_rate_init* y por *learning_rate* que hacen referencia al valor del módulo del desplazamiento y los cambios que se le pueden aplicar respectivamente. Así, un valor de *learning_rate* igual a *"constant"* mantendrá el módulo del paso constante, mientras que *"adaptive"* lo irá variando en función de la solución actual en cada iteración.

In [ ]:
solver = ____ # elige un optimizador
learning_rate_init = ____ # elige un paso
learning_rate = ____ # elegir la estrategia a seguir con el paso

El siguiente conjunto de parámetros hace referencia a las iteraciones del optimizador en el proceso de minimización. Éstos son:
- número máximo de iteraciones del optimizador: el optimizador realizará iteraciones hasta que converja o se alcance *max_iter*.
- *tol*: define lo que se entiende por convergencia
- *n_iter_no_change*: número máximo de iteraciones para que la función de pérdida disminuya


In [ ]:
max_iter = ____ # número máximo de iteraciones
tol = ____ # tolerancia
n_iter_no_change = ____ # númer máximo de épocas permitidas para que la función de pérdida disminuya

Finalmente, queda por difinir el conjunto de validación. Este conjunto se puede definir a mano y no especificar el siguiente conjunto de parámetros, sin embargo, el objeto *MLPRegressor* nos permite hacer el proceso de creación del conjunto de validación simplemente especificando un parámetro

In [ ]:
validation_fraction = ____ # fracción de elementos de entrada que se usarán para formar el conjunto de validación
early_stopping=True

### 2.3 Definición de la red

In [ ]:
net = MLPRegressor(
    activation=activation,
    hidden_layer_sizes=hidden_layer_sizes,
    solver=solver,
    learning_rate_init=learning_rate_init,
    learning_rate=learning_rate,
    max_iter=max_iter,
    tol=tol,
    n_iter_no_change=n_iter_no_change,
    validation_fraction=validation_fraction,
    verbose=True,
    early_stopping=early_stopping)

## 3 Entrenamiento

Una vez definidos los conjuntos de entrenamiento y los parámetros de la red, el entrenamiento se hace a través del método *fit()* que recibe como primer argumento el conjunto de datos de entrada para el entrenamiento y como segundo el conjunto de datos de salida con los que debe contrastar la propia salida de la red. El proceso de ajuste de la red neuronal consiste, precisamente, en comparar las salidas de la red con el conjunto de entrada y minimizar el error cuadrático medio

In [ ]:
# rellenar esta celda con la llamada a la función fit para entrenar la red


## 4 Mostrar y guardar los resultados

Cuando el entrenamiento a terminado los parámetros (b,w) - bias, weight - de la red se han ajustado de manera que ajustan la salida de la red al conjunto de datos *ytrain*. Estos valores se pueden llamar mediante los atributos *intercepts_* y *coefs_*

In [ ]:
# Mostrar por pantalla los parámetros b y w de la red nuronal


Para guardar la red neuronal ya entrenada vamos a utilizar la librería *pickle* de Python. Los comandos a ejecutar son:
```
output_file_name = "nombre de salida del fichero a guardar"
pickle.dump(nombre_de_la_red,open(output_file_name,"wb"))
```

In [ ]:
# Rellenar la celda con el código necesario para guardar la red




El valor de pérdida (el error cuadrático entre la salida de la red y los valores ytrain) de la red se puede obtener llamando al atributo *loss_* mientras que los valores para cada iteración se obtienen con el atributo *loss_curve_*

In [ ]:
# Muestra por pantalla el valor actual de pérdida y dibuja en una gráfica la curva de pérdida


In [ ]:
# Mustra por pantalla el score para el conjunto de entrenamiento. 
#¿Cómo compara con el valor mostrado en el entrenamiento con el del conjunto de validación?


## 5 Estudio de las incertidumbres

Una vez entrenada la red se utiliza el conjunto *test* para comprobar los resultados y caracterizar las incertidumbres. Dado un conjunto de espectros de entrada, la red asignará a cada espectro un valor de salida que corresponde con una temperatura, una gravedad y una metalicidad. Para obtener estos valores se utiliza el método *predict()* de la red neuronal.

Hay que recordar que la red fue entrenada con los objetivos (valores de salida) escalados y hay que desescalarlos.

In [ ]:
# Obtener los parámetros estelares que la red entrenada asocia a los espectros del conjunto test
# Recordar que los valores de los parámetros con los que se entrenó la red han sido escalados.

test_predictions = scaler.inverse_transform(nn.____(_____)) # rellenar los huecos con los valores para obtener las predicciones de la red asociadas a los espectros del conjunto test
test_values = # rellenar para obtener los valores de los parámetros reales del conjunto test

print(___) # rellenar para mostrar los valores de las predicciones

El error medio o *bias* para cada variable viene dado por el valor medio de la distribución *valores_de_la_red* - *valores_reales*. Para obtener estas distribuciones tenemos que acceder a los elementos de *test_predictions* y *test_values* que corresponden a cada parámetro astrofísico. Éstos están guardados en el orden en que los definimos (temperatura, gravedad, metalicidad).

Dado un *array* de *arrays*, si se quiere acceder al primer elemento de cada uno de los *arrays* se puede utilizar la siguiente nomenclatura:
```
primer_valor_cada_array = array_de_array[:,0]
```
Cambiando el índice se puede obtener todos los segundos, los terceros, etc.

In [ ]:
# Construir las distribuciones de predicciones menos valores reales
delta_teff = test_predictions[____] - test_values[___] # rellenar los huecos
delta_logg = ___ # rellenar
delta_mh = _____ # rellenar

In [ ]:
# Ilustrar en un gráfico las distribuciones de diferencias (deltas). Para ello pintar un histograma usando plt.hist(). 



In [ ]:
# en esta celda vamos a obtener los biases y las uncertidumbres.
# para obtenerlas hay que utilizar los métodos mean() y std() 
bias_teff = _____
bias_logg = _____
bias_mh = _____
uncert_teff = ____
uncert_logg = ____
uncert_mh = ____

In [ ]:
# mostrar los resultados de bias y uncert para cada variable


## 6 Nota final para el entrenamiento

Normalmente hay que realizar un estudio detallado para encontrar la configuración óptima de la red a través de los posibles valores de parámetros e hiperparámetros. La configuración óptima dependerá del tipo de problema que se quiere resolver, los datos de entrenamiento...; por ello es bueno cambiar la arquitectura y los valores de los parámetros y entrenar con los mismos datos.

Utiliza las celdas de debajo (tantas como necesites) para construir nuevas redes y comparar los resultados que dan cada una de ellas

## 7 Aplicación a espectros reales (medidos por RAVE)

Ahora se va a aplicar la red neuronal entrenada a datos reales. 
El fichero *data_spectra_input_school_v2_8420_8785_003899999999998727.pkl* contiene espectros medidos por el experimento [RAVE](https://www.rave-survey.org/) y, a partir de ellos, la red neuronal va a asignar a cada espectro una temperatura effectiva, un logaritmo de la gravedad y una metalicidad. Posteriormente, se mostrará en un gráfico las gravedades frente a las temperaturas. Así, el esquema de este análisis es:
- Lectura de datos: entender las diferencias entre los datos y las simulaciones
- Determinación de los parámetros astrofísicos
- Dibujar el diagrama de logg vs Teff (¿es, en tu caso, necesaria una corrección?)

### 7.1 Lectura de datos

In [ ]:
# rellenar el hueco para leer el fichero de datos
data = ____

In [ ]:
# pintar algunos espectros
idx = ___ 
plt.figure(figsize=(13,8))
plt.plot(____,____)
plt.xlabel(r"$\lambda /nm$")
plt.ylabel(r"$Normalized flux$")

### 7.2 Inferencia: determinación de parámetros astrofísicos

In [ ]:
data_specs = _____.______.tolist() # obtener los valores de los espctros. Nota: se convierte a una lista porque MLPRegressor.predict recibe como argumento una lista
data_preds = _____ # aplicar la red neuronal

In [ ]:
# Utiliza esta celda (y tantas como necesites) para explorar los resultados


### 7.3 Diagrama de inferencias

In [ ]:
# rellenar para dibujar el diagrama.
# Nota: el diagrama que se quiere dibujar debe tener los ejes invertidos. Si se quiere dibujar el eje X invertido hay que llamar a la instrucción plt.gca().invert_xaxis()




